In [2]:
import pandas as pd
import aesara_theano_fallback.tensor as tt
import exoplanet as xo

import pymc3 as pm
import pymc3_ext as pmx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
import pickle
import seaborn as sns
from astropy.io import ascii
from astropy.table import Table
from datetime import date

# Writing ASCII files for upload to CDS

We will have files for:

* Each CHEOPS visit (x4)
* Each MUSCAT2 bandpass (x4)
* Saint-Ex lightcurve (x1)
* LCO/McD lightcurve (x1)

which will include:
* raw flux
* meta data 
* detrended flux 
* best-fit models 
* 1/2-sigma errors on best-fit models

In [3]:
alldata = pickle.load(open("all_data_for_combined_model.pkl","rb"))

In [4]:
import glob
import gzip
import pickle

filename='P35_circ_ncbaenwgr_na_nogp_nocontam_fixld_35d_fixed_ror_vals_20211205'
save_loc="/Volumes/LUVOIR/Cheops_data/TOI2076/CombModelOutputs/"

bytes_in = bytearray(0)
max_bytes = 2**31 - 1
for file in glob.glob(save_loc+"TOI2076_finalmodel_"+filename+"_trace*gz"):
    with gzip.open(file, 'rb') as f_in:
        bytes_in += f_in.read(max_bytes)
trace = pickle.loads(bytes_in)

In [8]:
n=0
alldata['cheops_'+str(n)].keys()

dict_keys(['phi', 'flux', 'time', 'fine_time', 'norm_time', 'flux_err', 'phi_sorted', 'phi_sorting', 'time_sorting', 'smear', 'deltaT', 'bg', 'sin_phi', 'cos_phi'])

In [33]:
[i for i in 'griz']

['g', 'r', 'i', 'z']

In [44]:
data = []

allcols=[]

for scope in ['cheops_'+str(n) for n in range(4)]+[filt+'_lco' for filt in 'griz']+['r_sex', 'z_mcd']:
    chdata=Table()
    chdata['time']=alldata[scope]['time']
    for row in alldata[scope]:
        if len(alldata[scope][row])==len(alldata[scope]['time']) and 'sort' not in row and 'norm_' not in row and row!='time':
            chdata[row]=alldata[scope][row]
            if row not in allcols:
                allcols+=[row]
    for newrow in ['flux_cor','planets_x','summodel_x']:
        if newrow+'_'+scope in trace.varnames:
            if trace[newrow+'_'+scope].shape[1]==1:
                chdata[newrow+'_med']=np.tile(np.nanmedian(trace[newrow+'_'+scope],axis=0),len(alldata[scope]['time']))
                chdata[newrow+'_sd']=np.tile(np.nanstd(trace[newrow+'_'+scope],axis=0),len(alldata[scope]['time']))
            else:
                chdata[newrow+'_med']=np.nanmedian(trace[newrow+'_'+scope],axis=0)
                chdata[newrow+'_sd']=np.nanstd(trace[newrow+'_'+scope],axis=0)
            if newrow not in allcols:
                allcols+=[newrow]
        else:
            print(newrow)
    ascii.write(chdata, 'cdsdat/'+scope+'.dat', overwrite=True)
    data+=[chdata]

summodel_x
summodel_x
summodel_x


In [45]:
allcols

['phi',
 'flux',
 'flux_err',
 'smear',
 'deltaT',
 'bg',
 'sin_phi',
 'cos_phi',
 'flux_cor',
 'planets_x',
 'summodel_x',
 'airmass',
 'entropy',
 'width',
 'x',
 'y',
 'fwhm',
 'compcounts',
 'g/r',
 'r/i',
 'i/z']

In [41]:
for i in range(4):
    print(i,np.median(alldata['cheops_'+str(i)]['time']))

0 2348.133835280314
1 2334.040549950674
2 2333.5049420034047
3 2274.0856057913043


In [38]:
data[7].columns
# plt.plot(data[7]['summodel_x_z_lco_med'])
# plt.plot(data[7]['planets_x_z_lco_med'])
# plt.plot(data[7]['flux_cor_z_lco_med'])

<TableColumns names=('time','flux','flux_err','airmass','entropy','width','x','y','fwhm','compcounts','g/r','r/i','i/z','flux_cor_z_lco_med','flux_cor_z_lco_sd','planets_x_z_lco_med','planets_x_z_lco_sd','summodel_x_z_lco_med','summodel_x_z_lco_sd')>

In [5]:
trace.varnames

['Teff_interval__',
 'Rs_lowerbound__',
 'logg',
 'transit_times_0',
 'transit_times_1',
 'transit_times_2',
 'logror',
 'b_interval__',
 'u_star_cheops_interval__',
 'u_star_g_interval__',
 'u_star_i_interval__',
 'u_star_r_interval__',
 'u_star_tess_interval__',
 'u_star_z_interval__',
 'logs_phot_g_lco',
 'logs_phot_r_lco',
 'logs_phot_i_lco',
 'logs_phot_z_lco',
 'logs_phot_r_sex',
 'logs_phot_z_mcd',
 'logs_phot_cheops_0',
 'logs_phot_cheops_1',
 'logs_phot_cheops_2',
 'logs_phot_cheops_3',
 'logs_phot_tess',
 'dfdairmass_g_lco_interval__',
 'dfdentropy_g_lco_interval__',
 'dfdnorm_time_g_lco_interval__',
 'dfdwidth_g_lco_interval__',
 'dfdg/r_g_lco_interval__',
 'dfdr/i_g_lco_interval__',
 'd2fdairmass2_g_lco_interval__',
 'dfdairmass_r_lco_interval__',
 'dfdnorm_time_r_lco_interval__',
 'dfdwidth_r_lco_interval__',
 'dfdg/r_r_lco_interval__',
 'dfdr/i_r_lco_interval__',
 'd2fdairmass2_r_lco_interval__',
 'dfdairmass_i_lco_interval__',
 'dfdentropy_i_lco_interval__',
 'dfdnorm_ti

In [ ]:
data = Table()
data['x'] = np.array([1, 2, 3], dtype=np.int32)
data['y'] = data['x'] ** 2
ascii.write(data, 'values.dat', overwrite=True)